In [1]:
import pandas as pd
import numpy as np
import pulp
import os
import h5py
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform
from matchms.importing import load_from_mgf
from rdkit import Chem
from rdkit.Chem import DataStructs
from tqdm import tqdm
from myopic_mces.myopic_mces import MCES
import h5py


from massspecgym.utils import (
    morgan_fp, init_plotting, smiles_to_scaffold,
    train_val_test_split, create_split_file,
    MyopicMCES, MyopicMCESNew
    )

In [2]:
# Load the data from MGF file
file_path = '/Users/macbook/CODE/MS/data/MSn/MSn/20240914_msn_library_pos_all_lib_MSn.mgf'
spectra = list(load_from_mgf(file_path))

In [ ]:
# Extract unique SMILES
unique_smiles = set()
for spectrum in spectra:
    if 'smiles' in spectrum.metadata:
        unique_smiles.add(spectrum.metadata['smiles'])

In [5]:
len(unique_smiles)

14008

In [6]:
# Convert unique SMILES to a list
unique_smiles = list(unique_smiles)

In [ ]:
unique_smiles = unique_smiles[:100]

In [7]:
# Initialize the MCES computation instance
mces_calculator_new = MyopicMCESNew()

In [7]:
# Compute the MCES distance matrix
num_smiles = len(unique_smiles)
dists = np.zeros((num_smiles, num_smiles))

In [13]:
hdf5_path = '/Users/macbook/CODE/MS/data/MSn/MSn/mces_distances.hdf5'

# Initialize or load the distance matrix in HDF5 format
if os.path.exists(hdf5_path):
    hdf5_file = h5py.File(hdf5_path, 'a')
    dists = hdf5_file['mces'][:]
else:
    hdf5_file = h5py.File(hdf5_path, 'w')
    dists = np.full((num_smiles, num_smiles), np.inf)  # Initialize with infinity
    hdf5_file.create_dataset('mces', data=dists)


In [11]:
# Create a DataFrame to keep track of unique SMILES and their indices
smiles_df = pd.DataFrame(unique_smiles, columns=['smiles'])
smiles_df['index'] = smiles_df.index

# Save unique SMILES and indices to CSV for reference
smiles_df_path = '/Users/macbook/CODE/MS/data/MSn/MSn/unique_smiles.csv'
# smiles_df.to_csv(smiles_df_path, index=False)


In [14]:
for i in tqdm(range(num_smiles), desc="Computing MCES distances"):
    for j in range(i + 1, num_smiles):
        if np.isinf(dists[i, j]):  # Only compute if not already done
            dist = mces_calculator_new(unique_smiles[i], unique_smiles[j])
            dists[i, j] = dist
            dists[j, i] = dist
        # Save progress every 1000 calculations
        if (i * num_smiles + j) % 1000 == 0:
            hdf5_file['mces'][:] = dists
            hdf5_file.flush()


Computing MCES distances:   0%|          | 3/14008 [15:30<1207:15:40, 310.33s/it]


KeyboardInterrupt: 

In [ ]:
hdf5_file['mces'][:] = dists
hdf5_file.flush()
hdf5_file.close()

print("MCES distance computation completed and saved.")

In [2]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

# h5py

In [9]:
df = pd.read_csv('/Users/macbook/CODE/MS/data/MSn/MSn/MassSpecGym.tsv', sep='\t')

In [10]:
len(df)

231104

In [11]:
f = h5py.File('/Users/macbook/CODE/MS/data/MSn/MSn/all_smiles_mces.hdf5', 'r')
print(list(f.keys()))
dists = squareform(f['mces'])
dists_smiles = f['mces_smiles_order'][:].astype(str).tolist()
dists.shape, len(dists_smiles)


['mces', 'mces_smiles_order']


((34731, 34731), 34731)

In [13]:
len(dists_smiles)

34731

In [14]:
for s in df['smiles'].unique():
    assert s in dists_smiles

In [16]:
common_smiles = set(unique_smiles) & set(dists_smiles)

# Count how many unique_smiles are in dists_smiles
count = len(common_smiles)

print(f"Number of unique_smiles in dists_smiles: {count}")

Number of unique_smiles in dists_smiles: 62


In [17]:
# Create computation indices for MCES (all pairs, excluding self-comparisons)
computation_indices = []
for i in range(len(unique_smiles)):
    for j in range(i + 1, len(unique_smiles)):
        computation_indices.append([len(computation_indices), i, j])

# Save the input HDF5 file
with h5py.File('/Users/macbook/CODE/MS/data/MSn/MSn/mces_input.hdf5', 'w') as f:
    f.create_dataset('smiles', data=np.array(unique_smiles, dtype='S'))
    f.create_dataset('computation_indices', data=np.array(computation_indices, dtype='i'))

In [ ]:

# Verify that the distance matrix is symmetric
from scipy.linalg import issymmetric
assert issymmetric(dists)

In [ ]:
# Verify that the distance matrix is symmetric
from scipy.linalg import issymmetric
assert issymmetric(dists)

In [12]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [ ]:


# Prepare MCES or alternative distance matrix for clustering
# Here we create a placeholder distance matrix with random values
# Replace with your distance calculation logic
num_smiles = len(unique_smiles)
dists = np.random.rand(num_smiles, num_smiles)
dists = (dists + dists.T) / 2  # Symmetrize the matrix to ensure it's valid
np.fill_diagonal(dists, 0)  # Set diagonal to zero for self-distance

# Check if the matrix is symmetric
from scipy.linalg import issymmetric
assert issymmetric(dists)

# Perform Agglomerative Clustering using the precomputed distances
clustering = AgglomerativeClustering(
    metric='precomputed',
    linkage='single',
    distance_threshold=10,
    n_clusters=None
).fit(dists)

# Get cluster labels
clusters = clustering.labels_

# Create a DataFrame for the SMILES and their clusters
df = pd.DataFrame({'smiles': unique_smiles, 'cluster': clusters})

# Define a placeholder for collision energies and other metadata for stratification
# Here, we add mock data; replace with your actual data extraction logic
df['collision_energy'] = np.random.choice(['10.0', '20.0', '30.0', 'other'], size=len(df))
df['instrument_type'] = np.random.choice(['ITFT', 'QFT', 'Other'], size=len(df))
df['adduct'] = np.random.choice(['[M+H]+', '[M+Na]+'], size=len(df))

# Define stratification groups based on the metadata
def simple_ce(ce):
    if ce in ['10.0', '20.0', '30.0']:
        return ce
    return 'other'

def stratification_group(row):
    return str(row['adduct']) + str(row['instrument_type']) + str(simple_ce(row['collision_energy']))

df['stratification_group'] = df.apply(stratification_group, axis=1)

# Perform the train-validation-test split
X = df.index.values  # Use indices as identifiers
y = df['stratification_group'].values
groups = df['cluster'].values

sgkf = StratifiedGroupKFold(n_splits=3)
sgkf_split = sgkf.split(X, y, groups)

# Assign the splits to the DataFrame
folds_map = {}
for i, (train_index, test_index) in enumerate(sgkf_split):
    fold_name = ['train', 'val', 'test'][i]
    for idx in test_index:
        folds_map[idx] = fold_name

df['fold'] = df.index.map(folds_map)

# Show the split composition
def fold_composition(df_q, percentage=True):
    unique_counts = df_q.groupby(['fold']).agg({'smiles': 'nunique'})
    if not percentage:
        res = unique_counts
    else:
        total_counts = df_q.agg({'smiles': 'nunique'})
        percentage_counts = (unique_counts / total_counts) * 100
        res = percentage_counts.round(2)
    return res

print('Fold composition:')
print(fold_composition(df))

# Save the split data to a file
df.to_csv('/Users/macbook/CODE/MS/data/MSn/MSn/MassSpecGym_split.tsv', sep='\t', index=False)